In [1]:
import os
CORPUS_PATH = os.path.join('./documents/state_of_union')
filenames = sorted([os.path.join(CORPUS_PATH, fn) for fn in os.listdir(CORPUS_PATH)])

In [36]:
filenames[0]
f = open(filenames[0],'r')
test = f.read()
#print(test)
f.close()

In [37]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
#Our Document
filenames

# list for tokenized documents in loop
texts = []

# loop through document list
for i in filenames:
    
    # Open file
    file = open(i,'r')
    
    # clean and tokenize document text
    tokens = tokenizer.tokenize(file.read().lower())

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=2, id2word = dictionary, passes=20)

In [51]:
len(corpus)

65

In [67]:
# Function to display the words in each topic
def print_topic_words(ldamodel,n_topics,n_words):
    for i in ldamodel.print_topics(num_topics=n_topics, num_words=n_words):
        print(i[1])

In [52]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, passes=20)
print_topic_words(ldamodel,3,3)

[(0, '0.014*"will" + 0.010*"nation" + 0.009*"year"'), (1, '0.017*"will" + 0.012*"nation" + 0.010*"year"'), (2, '0.014*"will" + 0.013*"s" + 0.010*"american"')]


In [68]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=25, id2word = dictionary, passes=20)
print_topic_words(ldamodel,25,6)

0.006*"20th" + 0.006*"255" + 0.004*"underdevelop" + 0.003*"downturn" + 0.003*"midpoint" + 0.003*"ceonom"
0.019*"s" + 0.014*"will" + 0.011*"year" + 0.009*"america" + 0.009*"us" + 0.009*"american"
0.000*"s" + 0.000*"will" + 0.000*"american" + 0.000*"can" + 0.000*"nation" + 0.000*"peopl"
0.000*"will" + 0.000*"nation" + 0.000*"year" + 0.000*"s" + 0.000*"world" + 0.000*"must"
0.015*"will" + 0.011*"year" + 0.010*"nation" + 0.010*"program" + 0.008*"govern" + 0.007*"must"
0.000*"will" + 0.000*"year" + 0.000*"nation" + 0.000*"must" + 0.000*"american" + 0.000*"can"
0.000*"will" + 0.000*"nation" + 0.000*"must" + 0.000*"year" + 0.000*"s" + 0.000*"peopl"
0.016*"will" + 0.015*"nation" + 0.014*"world" + 0.013*"free" + 0.009*"can" + 0.009*"need"
0.000*"will" + 0.000*"year" + 0.000*"nation" + 0.000*"s" + 0.000*"must" + 0.000*"can"
0.000*"will" + 0.000*"year" + 0.000*"nation" + 0.000*"congress" + 0.000*"new" + 0.000*"must"
0.015*"will" + 0.013*"nation" + 0.011*"world" + 0.009*"must" + 0.008*"year" + 0.0